## <span style="color:#0B3B2E;float:right;font-family:Calibri">Jordan Graesser</span>

# SpFeas usage examples
---

## <span style="color:blue">SpFeas</span>
---

### Print help
> spfeas -h

### Print examples
> spfeas -e

### Print feature trigger options
> spfeas --options

### Parameters

- The input image (**`-i`**) should be given as the full path, image name, and extension.
- The band position (**`-bp`**) should be given as a list.
  - E.g., **`-bp 1`** or **`-bp 1 2 3`**
- Block size (**`--block`**)
  - Use a block size of 1 to compute a traditional, single pixel moving window.
  - Use a block size of 2 or greater (in even numbers) to compute a pixel block window.
  - Only use one block size per run (e.g., **`--block 2`**).
- Scales, or window sizes (**`--scales`**)
  - Multiple scales can be used.
  - Use even scales with an even **`--block`** and odd scales with **`--block 1`**.
- When a parameter default is **`False`** then it is a boolean trigger with no following values to give.
  - E.g., to use the visible spectrum in place of a single band, use **`--rgb`**.
- Be sure to declare the visible spectrum band order if using **`saliency`**.
  - E.g., **`--vis-order rgb`** or **`--vis-order bgr`**
- Triggers (**`-tr`**)
  - Multiple triggers can be used.
  - Separate triggers by a space.
- Spectral indices
  - Declare band positions when using **`gndvi`**, **`ndvi`**, or **`evi2`**.
  - E.g., **`--band-red 3`**

## Examples
---

### Compute the Mean with default settings
* Band=1; Block=2; Scales=8; Triggers='mean'
> spfeas -i /your_image.tif -o /your_output_directory

### Compute the Mean with custom settings
* Band=2; Block=4; Scales=8,16; Triggers='mean'
> spfeas -i /your_image.tif -o /your_output_directory -tr mean -bp 2 --block 8 --scales 8 16

### Compute the Mean & Histogram of Oriented Gradients with custom settings
* Band=2; Block=4; Scales=8,16; Triggers='mean', 'hog'
> spfeas -i /your_image.tif -o /your_output_directory -tr mean hog -bp 2 --block 8 --scales 8 16

### Compute the Mean & Histogram of Oriented Gradients with custom settings AND stack the results
* Band=2; Block=4; Scales=8,16; Triggers='mean', 'hog'
> spfeas -i /your_image.tif -o /your_output_directory -tr mean hog -bp 2 --block 8 --scales 8 16 --stack

### Compute the PanTex
* Band=2; Block=4; Scales=8,16; Triggers='pantex'
> spfeas -i /your_image.tif -o /your_output_directory -tr pantex -bp 2 --block 8 --scales 8 16

### Compute the Differential Morphological Profiles
* Band=2; Block=4; Scales=8,16; Triggers='dmp'
> spfeas -i /your_image.tif -o /your_output_directory -tr dmp -bp 2 --block 8 --scales 8 16

### Compute the Structural Feature Sets
* The **`sfs`** feature should be computed differently from all other features.
* The **`sfs`** feature will not automatically use an entire scale window, so there is no need to give multiple scales.
* Use a large scale. It is best to use the largest scale used in other features.
* Use a small block size and resample. 
* In this example, we are computing **`sfs`** with a block size of 2x2 at a scale of 128x128. After processing, the **`sfs`** features will be resampled to 16m x 16m cell size.
> spfeas -i /your_image.tif -o /your_output_directory -tr sfs --rgb --block 2 --scales 128 -sfs-rs 16

### Choosing input bands
* By default, **`SpFeas`** will use band 1 of the input image.
* If the input image is a 3-band (or more) image, with the visible spectrum in the first 3 bands, you can use the entire spectrum.
* Using the **`--rgb`** parameter tells **`SpFeas`** to take the average over the entire visible spectrum.
* If **`--rgb`** is called, it will replace the **`-bp`** parameter.
* In this example, we are computing the **`mean`** with the RGB average.
> spfeas -i /your_image.tif -o /your_output_directory -tr mean --rgb

### Parallel processing
* By default, **`SpFeas`** will use all availabe CPU cores to process image tiles (sections) in parallel.
* The number of tiles to process in parallel depends on your system's memory and the section size.
* In this example, we reduce the section size (in pixels) and use 2 parallel jobs.
> spfeas -i /your_image.tif -o /your_output_directory -tr mean --n-jobs 2 --sect-size 500

### Pyramid overviews
* The user might want to build pyramid layers to view larger image features.
* Use the **`--overviews`** parameter to create overview layers after the features are mosaicked.
> spfeas -i /your_image.tif -o /your_output_directory -tr mean hog dmp pantex --overviews

### Stacking processed features
* Use the same input image and output directory that were used to process the features.
* Choose the block and scale parameters that you wish to stack.
* In this command, any feature in the output directory that has been processed with a block size of 8x8 and scales of 8x8, 16x16, and 128x128 will be stacked. All other (if they exist) features will be left out.
> spfeas -i /your_image.tif -o /your_output_directory --rgb --block 8 --scales 8 16 128 --stack-only

## <span style="color:blue">Sampling</span>
---

The **`sample-raster`** tool comes from **`MpGlue`**. Use it to sample image features created with **`SpFeas`**.

### Print help
> sample-raster -h

### Print examples
> sample-raster -e

### Sample an image
* Use a polygon or point shapefile of land cover categories to sample an image or feature stack. Below, `-s` is the input shapefile, **`-i`** is the input image to sample, and **`-o`** is the output directory to save the sample text files to.
> sample-raster -s land_cover_samples.shp -i feature_image.vrt -o output_samples_dir

## <span style="color:blue">Image classification</span>
---

The classification tool, **`classify`**, also comes from **`MpGlue`**. It can be used to train classification models and predict class labels.

### Print help
> classify -h

### Print examples
> classify -e

### Train and save a model
* Below, we are training a Random Forest (`RF`) classifier with 100 `trees` and saving the model `--output-model` to file (RF_model.txt).
> classify -s /output_samples_dir/name_of_samples.txt --output-model /RF_model.txt --classifier-info "{'classifier': 'RF', 'trees': 100}"

### Classify an image
* Below, we use the model trained in the previous step to classify an image.
* Notice that we have now added the input image to classiy and the output mapped image.
* Also, we have now set `--input-model` instead of `--output-model`.
> classify -i feature_image.vrt -o output_map.tif -s output_samples_dir/name_of_samples.txt --input-model /RF_model.txt --classifier-info "{'classifier': 'RF'}"

### Train a model and classify an image
* Here we are training the model and classify an image in one step (previous two steps put together).
> classify -i feature_image.vrt -o output_map.tif -s output_samples_dir/name_of_samples.txt --output-model /RF_model.txt --classifier-info "{'classifier': 'RF'}"

## <span style="color:blue">Reclassification of thematic maps</span>
---

**`MpGlue`**'s reclassify tool can be used to reclassify thematic map values.

### Print help
> reclassify -h

### Print examples
> reclassify -e

### Reclassify a thematic map
* Here we want to reclassify all values of 10 to 9.
> reclassify -i /input_map.tif -o /reclassified_map.tif -r "{10:9}"

* In this example we are reclassifying 1 to 2 and 3 to 2.
> reclassify -i /input_map.tif -o /reclassified_map.tif -r "{1:2, 3:2}"

## <span style="color:blue">Recoding thematic maps</span>
---

* **`MpGlue`**'s **`recode`** tool is used when the user has specific areas of a thematic map that need reclassified (or cleaned).
* The user can specify polygons and recode rules for specific areas on a map.

### Print help
> recode -h

### Print examples
> recode -e

### Recode a thematic map
* In this example, we use a polygon shapefile with a set of recode rules.
* The **`-c`** parameter is used to tell the **`recode`** tool which polygon id column to use.
* The **`--rules`** tell the **`recode`** tool where and what to reclassify.
* Walking through the recode rules: 1) The first key positions (1 and 2) tell the **`recode`** tool which polygon feature to process; 2) The second set of key positions (e.g., {5:4} and {2:1, 3:4}) tell the **`recode`** rule which classes to reclassify.
* In feature 1, reclassify class 5 to 4.
* In feature 2, relcassify class 2 to 1, and reclassify 3 to 4.
> recode -p /polygon.shp -i /input_map.tif -o /reclassified_map.tif -c Id --rules "{1: {5:4}, 2: {2:1, 3:4}}"